In [5]:
%watermark
-mvp numpy,scipy,sklearn,matplotlib

UsageError: Line magic function `%watermark` not found.


In [1]:
# benchmark.py
import time
class benchmark(object):
    def __init__(self,name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self,ty,val,tb):
        end = time.time()
        print(f"{self.name} : {end-self.start:0.3f} seconds")
        return False

<hr/>

<h1>Memoization</h1>
<p>
<cite>Memoization is a way to lower a function's time cost in exchange for space cost; that is, memoized functions become optimized for speed in exchange for a higher use of computer memory space.</cite>
</p>

<h2>Manual implementation</h2>

Create a memoize fuction used as decorator and test on a funtion simulating long calculation<br>
Source: <a href="http://decorator.readthedocs.io/en/latest/tests.documentation.html">decorator documentation</a>

In [5]:
import functools
def memoize_uw(func):
    func.cache = {}

    def memoize(*args, **kw):
        if kw:  # frozenset is used to ensure hashability
            key = args, frozenset(kw.items())
        else:
            key = args
        if key not in func.cache:
            func.cache[key] = func(*args, **kw)
        return func.cache[key]
    return functools.update_wrapper(memoize, func)

In [6]:
import time

@memoize_uw
def f1(x):
    "Simulate some long computation"
    time.sleep(1)
    return x

On the second call, the benefit of caching is sensible.

In [7]:
with benchmark("f1(40), first pass"):
    f1(40)
with benchmark("f1(40), second pass"):
    f1(40)

f1(40), first pass : 1.001 seconds
f1(40), second pass : 0.000 seconds


<h2>functools.lru_cache</h2>
Python 3.2 introduces such a decorator in the <code><a href="https://docs.python.org/3/library/functools.html#functools.lru_cache">functools</a></code> module. Methods are available:<ul>
    <li><code>cache_clear()</code></li>
    <li><code>cache_info()</code></li>
    </ul>

In [1]:
from functools import lru_cache

@lru_cache(maxsize=32)
def f2(x):
    "Simulate some long computation"
    time.sleep(1)
    return x

In [2]:
with benchmark("f2(40), first pass"):
    f2(40)
with benchmark("f2(40), second pass"):
    f2(40)

NameError: name 'benchmark' is not defined

In [10]:
f2.cache_info()

CacheInfo(hits=1, misses=1, maxsize=32, currsize=1)

In [11]:
f2(40)

40

In [12]:
f2.cache_info()

CacheInfo(hits=2, misses=1, maxsize=32, currsize=1)

In [13]:
f2(50)

50

In [14]:
f2.cache_info()

CacheInfo(hits=2, misses=2, maxsize=32, currsize=2)

In [15]:
f2.__doc__

'Simulate some long computation'